In [1]:
import numpy as np
import os, sys, torch
sys.path.append("src")
from src.model import ShortChunkCNN_Res

wavs_path = "/data/avramidi/harmonix/audio_wav/"
song_file = os.listdir(wavs_path)[1]

In [3]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"
input_length = 73600
batch_size = 32
model = ShortChunkCNN_Res()
model.to(device)
model.eval()

# MSD model is not compatible !
model_load_path = "data/pretrained_models/focal_loss_all_data/"

state = torch.load(os.path.join(model_load_path, "best_model.pth"), map_location=device)
if "spec.mel_scale.fb" in state.keys():
    model.spec.mel_scale.fb = state["spec.mel_scale.fb"]
model.load_state_dict(state)

<All keys matched successfully>

In [7]:
npy_path = os.path.join("/data/avramidi/harmonix/npy_audios", song_file.replace(".wav", ".npy"))
raw = np.load(npy_path, mmap_mode="r")
hop = (len(raw) - input_length) // batch_size  # split chunk
input = [
    torch.Tensor(raw[i * hop : i * hop + input_length]).unsqueeze(0)
    for i in range(batch_size)
]
out = model(torch.vstack(input).to(device))

In [11]:
out[1]

tensor([[0.7860, 0.0000, 0.0000,  ..., 0.0000, 0.1943, 0.4205],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.6690, 0.4961],
        [0.9851, 0.0155, 0.0000,  ..., 0.0000, 0.0932, 0.1117],
        ...,
        [0.0000, 0.0000, 0.2108,  ..., 0.0000, 0.5265, 0.0000],
        [0.0000, 0.0000, 0.5789,  ..., 0.0000, 1.4831, 0.0000],
        [0.2108, 0.0000, 0.0420,  ..., 0.0000, 1.0663, 0.0000]],
       device='cuda:1', grad_fn=<CloneBackward0>)